In [1]:
import trimesh
import networkx
import torch
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm
from torch.utils.data import Dataset
from pathlib import Path

In [2]:
class STLDataset(Dataset):

    def __init__(self, root_path, tensor_size):
        self.stl_path = root_path
        self.stl_list = sorted(self._get_filenames(self.stl_path))
        self.tensor_size = tensor_size

    def __getitem__(self, idx):
        mesh = trimesh.load_mesh(self.stl_list[idx], force='mesh')
        adj = torch.from_numpy(networkx.adjacency_matrix(trimesh.graph.vertex_adjacency_graph(mesh)).toarray())
        adj = adj.float()
        triangles_num = torch.tensor(mesh.faces.shape[0])
        triangle_vertices_coords = self.fix_size_coords(torch.from_numpy(mesh.vertices))
        adj = self.fix_size_adj(adj, tensor_size=self.tensor_size)
        return adj, triangles_num, triangle_vertices_coords

    def __len__(self):
        return len(self.stl_list)
    
    @staticmethod
    def _get_filenames(path):
        return [f for f in path.iterdir() if f.is_file()]

    @staticmethod
    def fix_size_adj(input_tensor, tensor_size=42):
        if input_tensor.shape[0] < tensor_size:
            zeros = torch.zeros(input_tensor.shape[0], tensor_size - input_tensor.shape[0])
            tensor = torch.cat([input_tensor, zeros], dim=1)
    
            zeros = torch.zeros(tensor_size - input_tensor.shape[0], tensor_size)
            tensor = torch.cat([tensor, zeros], dim=0)
            return tensor
        elif input_tensor.shape[0] > tensor_size:
            return input_tensor[:tensor_size, :tensor_size]
        else:
            return input_tensor

    @staticmethod
    def fix_size_coords(input_tensor, tensor_size=42):
        if input_tensor.shape[0] < tensor_size:
            zeros = torch.zeros(tensor_size - input_tensor.shape[0], input_tensor.shape[1])
            tensor = torch.cat([input_tensor, zeros], dim=0)

            return tensor
        elif input_tensor.shape[0] > tensor_size:
            return input_tensor[:tensor_size]
        else:
            return input_tensor

In [3]:
input_dir = "../data/Thingi10K/models"
ADJACENCY_MATRIX_SIZE = 100

dataset = STLDataset(Path(input_dir), tensor_size=ADJACENCY_MATRIX_SIZE)

In [4]:
batch_size = 8
data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, pin_memory=True
)

In [5]:
for batch in data_loader:
    if not batch:
        continue
    adjacency_matrices, triangles_num, coordinates = batch
    if adjacency_matrices.dtype == torch.int32:
        KeyError("Fuck")
    print(adjacency_matrices)
    print(triangles_num)
    print(coordinates.shape)

tensor([[[0., 1., 1.,  ..., 0., 0., 0.],
         [1., 0., 1.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 1., 1.,  ..., 0., 0., 0.],
         [1., 0., 1.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 1., 1.,  ..., 0., 0., 0.],
         [1., 0., 1.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 1., 1.],
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 1., 0., 0.]],

        ...,

        [[0., 1., 1.,  ..., 0., 0., 0.],
         [1., 0., 1.,  ..., 0., 0., 0.],
         [1., 1., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0., 

### Initialise model.

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [7]:
class TriangleNet(nn.Module):
    """
    Regression model for predicting number of triangles
    based on the given adjacency matrix.

    adj_size: size of the adjacency matrix.
    """

    def __init__(self, adj_size: int = 42):
        super(TriangleNet, self).__init__()

        # TODO: fix too many neurons
        input_neurons = adj_size * adj_size
        hidden_s = input_neurons // 2

        self.layers = nn.Sequential(
            nn.Linear(input_neurons, hidden_s),
            nn.ReLU(),
            nn.Linear(hidden_s, 1)
        )

    def forward(self, x: torch.tensor):
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = self.layers(x)

        # round output
        # x = torch.round(x)
        # x = x.type(torch.long)
        return x.flatten()

# Params

In [8]:
EPOCHS = 1
triangle_net = TriangleNet(ADJACENCY_MATRIX_SIZE).to(device)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(triangle_net.parameters(), lr=0.001)
triangle_net

TriangleNet(
  (layers): Sequential(
    (0): Linear(in_features=10000, out_features=5000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=5000, out_features=1, bias=True)
  )
)

### Train loop

In [9]:
def train_one_epoch():
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, (adj, triangles_num, triangles_coords) in enumerate(data_loader):
        # Every data instance is an input + label pair
        triangles_num = triangles_num.type(dtype=torch.float32)

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # print(adj)
        # Make predictions for this batch
        outputs = triangle_net(adj)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, triangles_num)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.

    return last_loss

In [10]:
best_vloss = 1_000_000.

for epoch in tqdm(range(EPOCHS)):
    print('EPOCH {}:'.format(epoch + 1))

    # Make sure gradient tracking is on, and do a pass over the data
    triangle_net.train(True)
    avg_loss = train_one_epoch()

    print("Train loss: ", avg_loss)

    # running_vloss = 0.0
    # # Set the model to evaluation mode, disabling dropout and using population
    # # statistics for batch normalization.
    # model.eval()

    # # Disable gradient computation and reduce memory consumption.
    # with torch.no_grad():
    #     for i, vdata in enumerate(validation_loader):
    #         vinputs, vlabels = vdata
    #         voutputs = model(vinputs)
    #         vloss = loss_fn(voutputs, vlabels)
    #         running_vloss += vloss

    # avg_vloss = running_vloss / (i + 1)
    # print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

    # # Track best performance, and save the model's state
    # if avg_vloss < best_vloss:
    #     best_vloss = avg_vloss
    #     model_path = 'triangle_net_{}'.format(epoch)
    #     torch.save(triangle_net.state_dict(), model_path)

  0%|          | 0/1 [00:00<?, ?it/s]

EPOCH 1:


100%|██████████| 1/1 [01:35<00:00, 95.02s/it]

Train loss:  0.0


In [12]:
test_data = next(iter(data_loader))
test_data

[tensor([[[0., 1., 1.,  ..., 0., 0., 0.],
          [1., 0., 1.,  ..., 0., 0., 0.],
          [1., 1., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 1., 1.,  ..., 0., 0., 0.],
          [1., 0., 1.,  ..., 0., 0., 0.],
          [1., 1., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 1., 1.,  ..., 0., 0., 0.],
          [1., 0., 1.,  ..., 0., 0., 0.],
          [1., 1., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 1., 1.],
          [0., 0., 0.,  ..., 1., 0., 0.],
          [0., 0., 0.,  ..., 1., 0., 0.]],
 
         ...,
 
         [[0., 1., 1.,  ..., 0., 0., 0.],
          [1., 0., 1.,  ..., 0., 0., 0.],
          [1., 1., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 

In [14]:
triangle_net.eval()
with torch.no_grad():
    adj, triangles_num, triangles_coords = test_data
    outputs = triangle_net(adj)
    print(outputs)

tensor([461.7015, 358.1476, 753.6016, 538.3982, 897.3281, 470.2268, 840.6819,
         81.4737])
